In [1]:
from tensorflow import keras
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.datasets import fashion_mnist
from keras.layers import Conv2D, Activation, MaxPooling2D,Flatten,Dense
from keras.callbacks import ModelCheckpoint

In [2]:
(x_train,y_train), (x_test,y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 5s 1us/step


In [3]:
x_train = x_train.reshape(-1,28,28,1)
x_test=x_test.reshape(-1,28,28,1)
x_train = x_train/255
x_test = x_test/255

In [4]:
def cnn():
    model = keras.models.Sequential()
    model.add(Conv2D(32,(3,3),input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(32,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Flatten())
    
    model.add(Dense( 10))
    model.add(Activation('softmax'))
    
    model.compile(optimizer = 'adam',
                 loss = keras.losses.SparseCategoricalCrossentropy(),
                 metrics = ["accuracy"])
    return model

In [5]:
model = cnn()

In [6]:
checkpointer = ModelCheckpoint(filepath='best_mnist_model.hdf5', 
                               verbose=1, save_best_only=True)
model.fit(x_train,y_train,epochs=2, batch_size=128, verbose=1)

Epoch 1/2
469/469 [==============================] - 130s 278ms/step - loss: 0.6444 - accuracy: 0.7735
Epoch 2/2
469/469 [==============================] - 131s 280ms/step - loss: 0.4124 - accuracy: 0.8543


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 26, 26, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 11, 11, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0

In [8]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 11s 36ms/step - loss: 0.4173 - accuracy: 0.85090s - loss: 0.4194 - accura


[0.41734784841537476, 0.8508999943733215]

# trying kerastuner

In [13]:
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [23]:
def cnn_with_tuning(hyper_params):
    model = keras.models.Sequential()
    
    model.add(Conv2D(hyper_params.Int("input",min_value=32,max_value=128,step=32),(3,3),input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    for i in range(hyper_params.Int("c2_layer_n",1,4)):
        model.add(Conv2D(hyper_params.Int(f"conv2_with_{i}_units",min_value=32,max_value=128,step=32),(3,3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Flatten())
    
    model.add(Dense( 10))
    model.add(Activation('softmax'))
    
    model.compile(optimizer = 'adam',
                 loss = keras.losses.SparseCategoricalCrossentropy(),
                 metrics = ["accuracy"])
    return model

In [26]:
tuner = RandomSearch(cnn_with_tuning,objective = "val_accuracy",max_trials=2,directory='tuning_log')
tuner.search(x = x_train,
            y=y_train,
            epochs = 1,
            batch_size=64,
            validation_data=(x_test,y_test))

INFO:tensorflow:Reloading Oracle from existing project tuning_log\untitled_project\oracle.json


184/938 [====>.........................] - ETA: 0s - loss: 2.3124 - accuracy: 0.06 - ETA: 4:23 - loss: 2.2987 - accuracy: 0.07 - ETA: 5:56 - loss: 2.2881 - accuracy: 0.11 - ETA: 6:45 - loss: 2.2871 - accuracy: 0.10 - ETA: 7:09 - loss: 2.2848 - accuracy: 0.09 - ETA: 7:26 - loss: 2.2773 - accuracy: 0.10 - ETA: 7:43 - loss: 2.2683 - accuracy: 0.12 - ETA: 7:51 - loss: 2.2619 - accuracy: 0.14 - ETA: 7:56 - loss: 2.2506 - accuracy: 0.16 - ETA: 8:03 - loss: 2.2393 - accuracy: 0.16 - ETA: 8:11 - loss: 2.2225 - accuracy: 0.17 - ETA: 8:11 - loss: 2.2143 - accuracy: 0.18 - ETA: 8:13 - loss: 2.2110 - accuracy: 0.18 - ETA: 8:16 - loss: 2.2051 - accuracy: 0.18 - ETA: 8:18 - loss: 2.1919 - accuracy: 0.20 - ETA: 8:17 - loss: 2.1820 - accuracy: 0.20 - ETA: 8:22 - loss: 2.1643 - accuracy: 0.22 - ETA: 8:26 - loss: 2.1506 - accuracy: 0.23 - ETA: 8:38 - loss: 2.1322 - accuracy: 0.24 - ETA: 8:45 - loss: 2.1138 - accuracy: 0.25 - ETA: 9:05 - loss: 2.0971 - accuracy: 0.25 - ETA: 9:08 - loss: 2.0789 - accuracy

370/938 [==========>...................] - ETA: 5:52 - loss: 0.9740 - accuracy: 0.65 - ETA: 5:52 - loss: 0.9720 - accuracy: 0.65 - ETA: 5:51 - loss: 0.9698 - accuracy: 0.65 - ETA: 5:51 - loss: 0.9688 - accuracy: 0.65 - ETA: 5:50 - loss: 0.9660 - accuracy: 0.65 - ETA: 5:49 - loss: 0.9639 - accuracy: 0.65 - ETA: 5:49 - loss: 0.9627 - accuracy: 0.65 - ETA: 5:48 - loss: 0.9611 - accuracy: 0.65 - ETA: 5:48 - loss: 0.9599 - accuracy: 0.65 - ETA: 5:47 - loss: 0.9581 - accuracy: 0.65 - ETA: 5:46 - loss: 0.9556 - accuracy: 0.65 - ETA: 5:46 - loss: 0.9537 - accuracy: 0.65 - ETA: 5:45 - loss: 0.9516 - accuracy: 0.65 - ETA: 5:45 - loss: 0.9502 - accuracy: 0.65 - ETA: 5:44 - loss: 0.9481 - accuracy: 0.65 - ETA: 5:44 - loss: 0.9464 - accuracy: 0.66 - ETA: 5:43 - loss: 0.9448 - accuracy: 0.66 - ETA: 5:42 - loss: 0.9433 - accuracy: 0.66 - ETA: 5:42 - loss: 0.9421 - accuracy: 0.66 - ETA: 5:41 - loss: 0.9407 - accuracy: 0.66 - ETA: 5:41 - loss: 0.9384 - accuracy: 0.66 - ETA: 5:40 - loss: 0.9363 - accura

556/938 [================>.............] - ETA: 4:20 - loss: 0.7882 - accuracy: 0.71 - ETA: 4:20 - loss: 0.7876 - accuracy: 0.71 - ETA: 4:19 - loss: 0.7875 - accuracy: 0.71 - ETA: 4:19 - loss: 0.7878 - accuracy: 0.71 - ETA: 4:18 - loss: 0.7879 - accuracy: 0.71 - ETA: 4:18 - loss: 0.7876 - accuracy: 0.71 - ETA: 4:17 - loss: 0.7873 - accuracy: 0.71 - ETA: 4:17 - loss: 0.7866 - accuracy: 0.71 - ETA: 4:16 - loss: 0.7859 - accuracy: 0.71 - ETA: 4:16 - loss: 0.7854 - accuracy: 0.71 - ETA: 4:15 - loss: 0.7853 - accuracy: 0.71 - ETA: 4:15 - loss: 0.7846 - accuracy: 0.71 - ETA: 4:14 - loss: 0.7842 - accuracy: 0.71 - ETA: 4:14 - loss: 0.7840 - accuracy: 0.71 - ETA: 4:13 - loss: 0.7830 - accuracy: 0.71 - ETA: 4:13 - loss: 0.7823 - accuracy: 0.71 - ETA: 4:12 - loss: 0.7815 - accuracy: 0.71 - ETA: 4:12 - loss: 0.7813 - accuracy: 0.71 - ETA: 4:11 - loss: 0.7809 - accuracy: 0.71 - ETA: 4:11 - loss: 0.7806 - accuracy: 0.71 - ETA: 4:11 - loss: 0.7799 - accuracy: 0.71 - ETA: 4:10 - loss: 0.7796 - accura

742/938 [======================>.......] - ETA: 2:54 - loss: 0.7094 - accuracy: 0.74 - ETA: 2:54 - loss: 0.7091 - accuracy: 0.74 - ETA: 2:53 - loss: 0.7086 - accuracy: 0.74 - ETA: 2:53 - loss: 0.7082 - accuracy: 0.74 - ETA: 2:52 - loss: 0.7081 - accuracy: 0.74 - ETA: 2:52 - loss: 0.7079 - accuracy: 0.74 - ETA: 2:51 - loss: 0.7077 - accuracy: 0.74 - ETA: 2:51 - loss: 0.7073 - accuracy: 0.74 - ETA: 2:50 - loss: 0.7067 - accuracy: 0.74 - ETA: 2:50 - loss: 0.7062 - accuracy: 0.74 - ETA: 2:49 - loss: 0.7058 - accuracy: 0.74 - ETA: 2:49 - loss: 0.7050 - accuracy: 0.74 - ETA: 2:49 - loss: 0.7045 - accuracy: 0.74 - ETA: 2:48 - loss: 0.7042 - accuracy: 0.74 - ETA: 2:48 - loss: 0.7039 - accuracy: 0.74 - ETA: 2:47 - loss: 0.7035 - accuracy: 0.74 - ETA: 2:47 - loss: 0.7032 - accuracy: 0.74 - ETA: 2:46 - loss: 0.7027 - accuracy: 0.74 - ETA: 2:46 - loss: 0.7023 - accuracy: 0.74 - ETA: 2:45 - loss: 0.7019 - accuracy: 0.74 - ETA: 2:45 - loss: 0.7015 - accuracy: 0.74 - ETA: 2:44 - loss: 0.7017 - accura

...

558/938 [================>.............] - ETA: 2:39 - loss: 0.8189 - accuracy: 0.70 - ETA: 2:39 - loss: 0.8187 - accuracy: 0.70 - ETA: 2:38 - loss: 0.8183 - accuracy: 0.70 - ETA: 2:38 - loss: 0.8174 - accuracy: 0.70 - ETA: 2:38 - loss: 0.8169 - accuracy: 0.70 - ETA: 2:38 - loss: 0.8165 - accuracy: 0.70 - ETA: 2:37 - loss: 0.8156 - accuracy: 0.70 - ETA: 2:37 - loss: 0.8153 - accuracy: 0.70 - ETA: 2:37 - loss: 0.8144 - accuracy: 0.70 - ETA: 2:36 - loss: 0.8137 - accuracy: 0.70 - ETA: 2:36 - loss: 0.8132 - accuracy: 0.70 - ETA: 2:36 - loss: 0.8126 - accuracy: 0.70 - ETA: 2:35 - loss: 0.8127 - accuracy: 0.70 - ETA: 2:35 - loss: 0.8127 - accuracy: 0.70 - ETA: 2:35 - loss: 0.8119 - accuracy: 0.70 - ETA: 2:34 - loss: 0.8108 - accuracy: 0.70 - ETA: 2:34 - loss: 0.8102 - accuracy: 0.70 - ETA: 2:34 - loss: 0.8100 - accuracy: 0.70 - ETA: 2:33 - loss: 0.8088 - accuracy: 0.70 - ETA: 2:33 - loss: 0.8080 - accuracy: 0.70 - ETA: 2:33 - loss: 0.8077 - accuracy: 0.70 - ETA: 2:32 - loss: 0.8071 - accura

744/938 [======================>.......] - ETA: 1:44 - loss: 0.7329 - accuracy: 0.73 - ETA: 1:44 - loss: 0.7322 - accuracy: 0.73 - ETA: 1:44 - loss: 0.7322 - accuracy: 0.73 - ETA: 1:44 - loss: 0.7318 - accuracy: 0.73 - ETA: 1:43 - loss: 0.7318 - accuracy: 0.73 - ETA: 1:43 - loss: 0.7313 - accuracy: 0.73 - ETA: 1:43 - loss: 0.7309 - accuracy: 0.73 - ETA: 1:42 - loss: 0.7306 - accuracy: 0.73 - ETA: 1:42 - loss: 0.7304 - accuracy: 0.73 - ETA: 1:42 - loss: 0.7302 - accuracy: 0.73 - ETA: 1:42 - loss: 0.7297 - accuracy: 0.73 - ETA: 1:41 - loss: 0.7292 - accuracy: 0.73 - ETA: 1:41 - loss: 0.7285 - accuracy: 0.73 - ETA: 1:41 - loss: 0.7281 - accuracy: 0.73 - ETA: 1:40 - loss: 0.7275 - accuracy: 0.73 - ETA: 1:40 - loss: 0.7270 - accuracy: 0.73 - ETA: 1:40 - loss: 0.7270 - accuracy: 0.73 - ETA: 1:39 - loss: 0.7268 - accuracy: 0.73 - ETA: 1:39 - loss: 0.7265 - accuracy: 0.73 - ETA: 1:39 - loss: 0.7263 - accuracy: 0.73 - ETA: 1:39 - loss: 0.7260 - accuracy: 0.73 - ETA: 1:38 - loss: 0.7258 - accura

931/938 [============================>.] - ETA: 52s - loss: 0.6739 - accuracy: 0.756 - ETA: 52s - loss: 0.6735 - accuracy: 0.756 - ETA: 52s - loss: 0.6731 - accuracy: 0.756 - ETA: 51s - loss: 0.6732 - accuracy: 0.756 - ETA: 51s - loss: 0.6732 - accuracy: 0.756 - ETA: 51s - loss: 0.6727 - accuracy: 0.756 - ETA: 51s - loss: 0.6723 - accuracy: 0.756 - ETA: 50s - loss: 0.6720 - accuracy: 0.757 - ETA: 50s - loss: 0.6718 - accuracy: 0.757 - ETA: 50s - loss: 0.6714 - accuracy: 0.757 - ETA: 50s - loss: 0.6712 - accuracy: 0.757 - ETA: 49s - loss: 0.6707 - accuracy: 0.757 - ETA: 49s - loss: 0.6705 - accuracy: 0.757 - ETA: 49s - loss: 0.6702 - accuracy: 0.757 - ETA: 48s - loss: 0.6701 - accuracy: 0.757 - ETA: 48s - loss: 0.6697 - accuracy: 0.757 - ETA: 48s - loss: 0.6694 - accuracy: 0.757 - ETA: 48s - loss: 0.6693 - accuracy: 0.757 - ETA: 47s - loss: 0.6690 - accuracy: 0.758 - ETA: 47s - loss: 0.6688 - accuracy: 0.758 - ETA: 47s - loss: 0.6688 - accuracy: 0.758 - ETA: 47s - loss: 0.6685 - accurac

938/938 [==============================] - ETA: 1s - loss: 0.6371 - accuracy: 0.76 - ETA: 1s - loss: 0.6368 - accuracy: 0.76 - ETA: 1s - loss: 0.6366 - accuracy: 0.76 - ETA: 0s - loss: 0.6363 - accuracy: 0.76 - ETA: 0s - loss: 0.6363 - accuracy: 0.76 - ETA: 0s - loss: 0.6362 - accuracy: 0.76 - ETA: 0s - loss: 0.6360 - accuracy: 0.76 - 268s 286ms/step - loss: 0.6360 - accuracy: 0.7696 - val_loss: 0.4753 - val_accuracy: 0.8320


INFO:tensorflow:Oracle triggered exit


In [29]:
print(tuner.get_best_hyperparameters()[0].values)
print(tuner.results_summary())
print(tuner.get_best_models()[0].summary())

{'input': 64, 'c2_layer_n': 2, 'conv2_with_0_units': 32, 'conv2_with_1_units': 96}


None
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
activation (Activation)      (None, 26, 26, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        18464     
_________________________________________________________________
activation_1 (Activation)    (None, 11, 11, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 96)      